In [1]:
import boto3
import os, yaml
import pandas as pd
os.getcwd()

'/home/wwh9345/Downloads/reddit-lake-starter'

In [2]:
from etl.utils import *

# s3 = s3_client(layer="bronze")
# s3.list_objects(Bucket="reddit-bronze")
# s3

In [4]:
cfg = load_yaml('./config/storage.yaml')
layout = cfg['layout']['bronze']
display(f"Config layout: {layout}")
key = layout.format(YYYY='2008', MM='01', type='rs')
print(f"example key: {key}")

'Config layout: year={YYYY}/month={MM}/{type}-{YYYY}-{MM}.zst'

example key: year=2008/month=01/rs-2008-01.zst


In [5]:
object_exists(
    s3, 
    bucket='reddit-bronze', 
    key=key)

False

In [ ]:
# ingest_upload.py
from botocore.exceptions import ClientError
from utils import object_exists, load_yaml, s3_client, parse_date_from_name, infer_type

...
cfg = load_yaml('./config/storage.yaml')
layout = cfg['layout']['bronze']
key = layout.format(YYYY=y, MM=m, type=t)

if object_exists(s3, args.bucket, key):
    print(f"SKIP: {f} (already exists in bucket)")
    continue

print(f"Uploading {f} -> s3://{args.bucket}/{key}")
s3.upload_file(str(f), args.bucket, key, ExtraArgs={...})


## ADMIN: Downloading older bucket (rs-2015-05) items for bucket deletion

In [4]:
def s3_client2(): # use this instead if all in same bucket
    endpoint = ENDPT_2
    key = KEY_2
    secret = SECRET_2
    validate_s3_env_vars(endpoint, key, secret)
    session = boto3.session.Session()
    return session.client("s3",
        endpoint_url=endpoint,
        aws_access_key_id=key,
        aws_secret_access_key=secret,
        config=Config(s3={"addressing_style":"path"})
    )

In [5]:
# callback function to track progress
def progress_callback(bytes_transferred):
    global total_bytes_downloaded
    total_bytes_downloaded += bytes_transferred
    print(f"Downloaded {total_bytes_downloaded / (1024 ** 3)} GB so far...")

# Initialize a counter for total bytes downloaded
total_bytes_downloaded = 0

bucket_name = 'rs-2015-05'
s3 = s3_client2()
list_objs = s3.list_objects(Bucket=bucket_name)

# obtain full list of bucket items to download
downloads_list = [list_objs.get('Contents','')[i]['Key'] for i in range(len(list_objs.get('Contents', [])))]
# add prefix string to create 'folder' in download destination
prefix_str = bucket_name + '_bucket' + '/'
# organise in dict for ease of downloading
downloads_dict = {file: prefix_str + file for file in downloads_list}
downloads_dict.pop('RC_2015-05.bz2', None)
downloads_dict.pop('RC_2015-05_10M_comments.csv', None)

for object_key, local_filename in downloads_dict.items():
    try:
        # Get object metadata to determine total size (optional, but useful for percentage calculation)
        response = s3.head_object(Bucket=bucket_name, Key=object_key)
        total_size_gb = int(response['ContentLength']) / (1024 ** 3)  # Convert bytes to GB
        print(f"Downloading '{object_key}' ({total_size_gb:.2f} GB)")

        # Download the file with the progress callback
        s3.download_file(
            Bucket=bucket_name,
            Key=object_key,
            Filename=object_key,
            Callback=progress_callback
        )
        print(f"Download of '{object_key}' complete to '{object_key}'")

    except Exception as e:
        print(f"Error during download: {e}")

Downloaded 0.000244140625 GB so far...
Downloaded 0.00048828125 GB so far...
Downloaded 0.000732421875 GB so far...
Downloaded 0.0009765625 GB so far...
Downloaded 0.001220703125 GB so far...
Downloaded 0.00146484375 GB so far...
Downloaded 0.001708984375 GB so far...
Downloaded 0.001953125 GB so far...
Downloaded 0.002197265625 GB so far...
Downloaded 0.00244140625 GB so far...
Downloaded 0.002685546875 GB so far...
Downloaded 0.0029296875 GB so far...
Downloaded 0.003173828125 GB so far...
Downloaded 0.00341796875 GB so far...
Downloaded 0.003662109375 GB so far...
Downloaded 0.00390625 GB so far...
Downloaded 0.004150390625 GB so far...
Downloaded 0.00439453125 GB so far...
Downloaded 0.004638671875 GB so far...
Downloaded 0.0048828125 GB so far...
Downloaded 0.005126953125 GB so far...
Downloaded 0.00537109375 GB so far...
Downloaded 0.005615234375 GB so far...
Downloaded 0.005859375 GB so far...
Downloaded 0.006103515625 GB so far...
Downloaded 0.00634765625 GB so far...
Downloade

In [17]:
bucket_name = 'rs-2015-05'
s3 = s3_client2()
list_objs = s3.list_objects(Bucket=bucket_name)

# obtain full list of bucket items to download
downloads_list = [list_objs.get('Contents','')[i]['Key'] for i in range(len(list_objs.get('Contents', [])))]
# add prefix string to create 'folder' in download destination
prefix_str = bucket_name + '_bucket' + '/'
# organise in dict for ease of downloading
downloads_dict = {file: prefix_str + file for file in downloads_list}

In [10]:
import os, threading
from tqdm.auto import tqdm

In [36]:
import os, threading
from tqdm.auto import tqdm

# 1) Page through the whole bucket (handles >1000 keys)
paginator = s3.get_paginator("list_objects_v2")
pages = paginator.paginate(Bucket=bucket_name)#, Prefix=prefix_str)

objects = []
total_bytes = 0
for page in pages:
    print('Page Items:')
    # print(page)
    for obj in page.get("Contents", []):
        objects.append(obj)              # obj has Key, Size, etc.
        total_bytes += obj["Size"]       # bytes
        print(obj["Size"], obj['Key'])
    print('\n')

# 2) Global progress bar in BYTES
lock = threading.Lock()
with tqdm(total=total_bytes, unit="B", unit_scale=True, unit_divisor=1024,
          desc="All downloads", dynamic_ncols=True) as global_pbar:
    for obj in objects:
        key = obj["Key"]
        size = obj["Size"]
        local_path = os.path.join(f"{bucket_name}_bucket", key)
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        # Per-file callback that updates the single global bar (thread-safe)
        def make_cb():
            def _cb(bytes_amount):
                with lock:
                    global_pbar.update(bytes_amount)
            return _cb

        s3.download_file(
            Bucket=bucket_name,
            Key=key,
            Filename=local_path,
            Callback=make_cb(),
            Config=Config(  # optional: tune chunk sizes to reduce callback frequency
                retries={"max_attempts": 10, "mode": "standard"}
            )
        )


Page Items:
5658889634 RC_2015-05.bz2
3144249675 RC_2015-05_10M_comments.csv
2024282239 RS_2015-05_full_submissions.parquet




All downloads:   0%|          | 0.00/10.1G [00:00<?, ?B/s]


AttributeError: 'Config' object has no attribute 'preferred_transfer_client'

In [ ]:
# # Initialize a session using Amazon S3
# s3 = boto3.client(
#     's3', 
#     aws_access_key_id='YOUR_ACCESS_KEY', 
#     aws_secret_access_key='YOUR_SECRET_KEY')

# # Bucket and file name
# bucket_name = 'your-bucket-name'
# file_key = 'your-dataset-key'

# # S3 URL
# s3_url = f"s3://{bucket_name}/{file_key}"

# # Using pandas to read a CSV file directly
# df = pd.read_csv(s3_url)


ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.

In [ ]:
# def load_layout(layer:str) -> str:
#     with open("config/storage.yaml", "r") as f:
#         # Optional: expand ${ENV_VAR} in YAML
#         text = os.path.expandvars(f.read())
#     cfg = yaml.safe_load(text)
#     return cfg["layout"][layer]

# display(load_layout('bronze'))
# load_layout('bronze').format(YYYY='2222', MM='08', type='rs')

'bronze/year={YYYY}/month={MM}/{type}-{YYYY}-{MM}.zst'

'bronze/year=2222/month=08/rs-2222-08.zst'